In [263]:
import mne
import numpy as np
import pandas as pd
from mne.preprocessing.nirs import beer_lambert_law
from mne.preprocessing.nirs import optical_density
from sklearn.preprocessing import MinMaxScaler
import sched, time, socket, struct
import os
import pickle

In [264]:
snirf_path = './task/2023-06-08_004.snirf'
raw_intensity = mne.io.read_raw_snirf(snirf_path, verbose=True, preload=False)
sample_freq = 0.5

# resample to one Hz
raw_intensity.load_data().resample(sample_freq, npad="auto")

Loading /Users/macintosh/Desktop/horse/fNIRS/tactile_brain_project/replay/task/2023-06-08_004.snirf
Reading 0 ... 4271  =      0.000 ...   419.839 secs...


<RawSNIRF | 2023-06-08_004.snirf, 28 x 210 (418.0 s), ~183 kB, data loaded>

In [265]:
# Define the motor unit to channel mapping
mu_0 = ['S5_D3 hbo', 'S5_D6 hbo']
mu_1 = ['S8_D6 hbo']
mu_2 = ['S8_D6 hbo']
mu_3 = ['S6_D5 hbo', 'S7_D5 hbo', 'S6_D7 hbo', 'S7_D4 hbo']
mu_4 = ['S6_D5 hbo', 'S7_D5 hbo', 'S6_D7 hbo', 'S7_D4 hbo']
mu_12 = ['S3_D1 hbo', 'S3_D2 hbo', 'S4_D2 hbo', 'S1_D1 hbo', 'S1_D2 hbo']
mu_13 = ['S3_D1 hbo', 'S3_D2 hbo', 'S4_D2 hbo', 'S1_D1 hbo', 'S1_D2 hbo']
mu_14 = ['S2_D3 hbo']
mu_15 = ['S5_D3 hbo', 'S2_D3 hbo']


In [266]:
raw_od = optical_density(raw_intensity)
raw_haemo = beer_lambert_law(raw_od, ppf=0.1)
ch_names = raw_haemo.info['ch_names']
# ch_names = ['0'] + ch_names

# Converts the snirf data into a CSV
data = pd.DataFrame(raw_haemo.get_data())

#Normalize the data
np_data = data.to_numpy()
scaler = MinMaxScaler(feature_range=(0, 200))
norm_data = scaler.fit_transform(np_data)
df = pd.DataFrame(norm_data)

# Add the channel names
df.insert(0, 'ch_names', ch_names)
df.to_csv('hemo_data.csv')


# Map channels to motor units

In [267]:

# Motor unit 0
mu_0_mask = df['ch_names'].isin(mu_0)
mu_0_df = df[mu_0_mask]

# Motor unit 1
mu_1_mask = df['ch_names'].isin(mu_1)
mu_1_df = df[mu_1_mask]

# Motor Unit 2
mu_2_mask = df['ch_names'].isin(mu_2)
mu_2_df = df[mu_2_mask]

# Motor Unit 3
mu_3_mask = df['ch_names'].isin(mu_3)
mu_3_df = df[mu_3_mask]

# Motor Unit 4
mu_4_mask = df['ch_names'].isin(mu_4)
mu_4_df = df[mu_4_mask]

# Motor Unit 12
mu_12_mask = df['ch_names'].isin(mu_12)
mu_12_df = df[mu_12_mask]

# Motor Unit 13
mu_13_mask = df['ch_names'].isin(mu_13)
mu_13_df = df[mu_13_mask]

# Motor Unit 14
mu_14_mask = df['ch_names'].isin(mu_14)
mu_14_df = df[mu_14_mask]

# Motor Unit 15
mu_15_mask = df['ch_names'].isin(mu_15)
mu_15_df = df[mu_15_mask]

print(mu_14_df)

    ch_names           0          1          2          3          4  \
2  S2_D3 hbo  157.995241  81.527345  25.251558  54.140197  42.419204   

           5          6          7          8  ...         200         201  \
2  43.801333  40.646814  23.685182  44.183025  ...  104.837514  110.829445   

          202         203         204        205        206         207  \
2  134.834555  147.618408  176.325528  163.06706  179.47073  125.958414   

          208         209  
2  141.169888  169.884826  

[1 rows x 211 columns]


In [273]:
#send signal to esp32 by chatgpt
# Arduino IP address and port
ip_address = '192.168.4.1'  # Replace with the IP address of your Arduino
port = 80


for id, column in enumerate(df):
    column_nums = len(df.columns)
    if column_nums == id:
        break
    print('tick', id)
    print(len(df.columns))
    mu_0_mean = np.int32(mu_0_df[id].mean())
    mu_1_mean = np.int32(mu_1_df[id].mean())
    mu_2_mean = np.int32(mu_2_df[id].mean())
    mu_3_mean = np.int32(mu_3_df[id].mean())
    mu_4_mean = np.int32(mu_4_df[id].mean())
    mu_12_mean = np.int32(mu_12_df[id].mean())
    mu_13_mean = np.int32(mu_13_df[id].mean())
    mu_14_mean = np.int32(mu_14_df[id].mean())
    mu_15_mean = np.int32(mu_15_df[id].mean())

    # Signal intensities (values between 0 and 255)
    intensities = [mu_0_mean, mu_1_mean, mu_2_mean, mu_3_mean, mu_4_mean, 0, 0, 0, 0, 0, 0, 0, mu_12_mean, mu_13_mean, mu_14_mean, mu_15_mean]
    print(intensities)
    # Create a TCP/IP socket
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    try:
        # Connect to the Arduino server
        sock.connect((ip_address, port))

        # Prepare the signal as a byte array
        signal = bytes(intensities)

        # Send the signal to the Arduino
        sock.sendall(signal)
        #intensities0 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
        #signal = bytes(intensities0)
        #sock.sendall(signal)

    finally:
        # Close the socket
        intensities0 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
        signal = bytes(intensities0)
        sock.sendall(signal)
        sock.close()
    
    time.sleep(sample_freq)

tick 0
211
[163, 0, 0, 177, 177, 0, 0, 0, 0, 0, 0, 0, 175, 175, 157, 166]
tick 1
211
[100, 0, 0, 157, 157, 0, 0, 0, 0, 0, 0, 0, 149, 149, 81, 98]
tick 2
211
[54, 11, 11, 119, 119, 0, 0, 0, 0, 0, 0, 0, 102, 102, 25, 67]
tick 3
211
[54, 30, 30, 123, 123, 0, 0, 0, 0, 0, 0, 0, 105, 105, 54, 81]
tick 4
211
[52, 43, 43, 123, 123, 0, 0, 0, 0, 0, 0, 0, 106, 106, 42, 73]
tick 5
211
[52, 44, 44, 119, 119, 0, 0, 0, 0, 0, 0, 0, 100, 100, 43, 74]
tick 6
211
[54, 51, 51, 124, 124, 0, 0, 0, 0, 0, 0, 0, 107, 107, 40, 74]
tick 7
211
[51, 44, 44, 111, 111, 0, 0, 0, 0, 0, 0, 0, 102, 102, 23, 63]
tick 8
211
[53, 54, 54, 127, 127, 0, 0, 0, 0, 0, 0, 0, 129, 129, 44, 75]
tick 9
211
[38, 33, 33, 86, 86, 0, 0, 0, 0, 0, 0, 0, 119, 119, 30, 53]
tick 10
211
[28, 23, 23, 62, 62, 0, 0, 0, 0, 0, 0, 0, 112, 112, 22, 40]
tick 11
211
[23, 20, 20, 54, 54, 0, 0, 0, 0, 0, 0, 0, 106, 106, 21, 34]
tick 12
211
[20, 18, 18, 42, 42, 0, 0, 0, 0, 0, 0, 0, 101, 101, 19, 30]
tick 13
211
[20, 13, 13, 35, 35, 0, 0, 0, 0, 0, 0, 0, 95

KeyboardInterrupt: 

In [272]:
# stop all vibration
host = "192.168.4.1"
port = 80
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.setsockopt(socket.IPPROTO_TCP, socket.TCP_NODELAY, 1)
s.connect((host, port))
for x in range(16):
        MESSAGE = struct.pack('d',0)
        print(MESSAGE)
        s.sendall(MESSAGE)
s.close()

b'\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x00\x00\x00\x00\x00\x00\x00\x00'
b'\x00\x00\x00\x00\x00\x00\x00\x00'
